In [3]:
import pandas as pd
import numpy as np
from itertools import product

import seaborn as sns


testcsv = '../../data/kaggle_files/test.csv'
traincsv = '../../data/cleaned/ames_clean.csv'

In [4]:
df_test = pd.read_csv(testcsv, index_col='Id')
df_train = pd.read_csv(traincsv, index_col='id')
df_test.shape

(879, 79)

In [5]:
df_test.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
Id,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [6]:
df_test.columns = df_test.columns.map(lambda col: col.replace(' ','_').lower())
df_test.index.name = 'id'

Let's drop the PID.

In [7]:
df_test.drop('pid', axis=1, inplace=True)

The `ms_subclass` column should be an object, the numbers are just weird codes for various styles of home. This is not a persistent change in the sense that it won't change what happens when the saved CSV is read by pandas, ie I'll have to do this change again when reading the file back into a dataframe, but I'll do it here so I remember to in the future.

In [8]:
df_test['ms_subclass'] = df_test['ms_subclass'].astype('object').map(str)
df_train['ms_subclass'] = df_train['ms_subclass'].astype('object').map(str)

Some columns encode the same information. For example there are a number of columns encoding information about the garage. Accoding to the documentation these columns are supposed to be be empty when there is no garage present. So if all of these columns are empty, we'll fill them with a string declaring that that indicates that the garage is missing. 

However if some of these columns have values but some do not, for example `Garage Finish` has a value but `Garage Cond` does not, we will drop that row. Some missing entries and some present indicates an error in the row. I'm going to ignore `Garage Yr Blt` for now and handle it at the end since I'm not sure what value it should actually take.

Fix the `pool_qc`, `misc_feature`, `alley`, `fence`, `fireplace_qe`

In [9]:
df_test['pool_qc'].fillna(value='No Pool', inplace=True)
df_test['misc_feature'].fillna(value='No Misc Feature', inplace=True)
df_test['alley'].fillna(value='No Alley', inplace=True)
df_test['fence'].fillna(value='No Fence', inplace=True)
df_test['fireplace_qu'].fillna(value='No Fireplace', inplace=True)

In [10]:
df_test.isnull().sum().sort_values(ascending=False).head()

lot_frontage     160
garage_yr_blt     45
garage_finish     45
garage_qual       45
garage_cond       45
dtype: int64

In [11]:
#keeps rows with no missing info, keeps rows which are missing every entry in the specified columns
#drops all the other rows, fills the missing entries with the new_value

#this is the same as in the training data except we're no longer dropping rows with contadictory data in these columns
#so now we're just filling the rows with all nan in garage/basement

def frame_column_filler(df, columns, new_value, inplace=False):
    if not inplace:
        tmp_df = df.copy()
    else:
        tmp_df = df
    
    num_nan = tmp_df[columns].isnull().sum(1)
    
    all_nan = set(num_nan[num_nan.apply(lambda x: x == len(columns))].index)
    no_nan = set(num_nan[num_nan.apply(lambda x: x == 0)].index)
    some_missing = {x for x in tmp_df.index if x not in all_nan.union(no_nan)}
    
    
    for index in tmp_df.index:
        if index in all_nan:
            for col in columns:
                tmp_df.at[index, col] = new_value
        
    if not inplace:
        return tmp_df

In [12]:
garage_cols = ['garage_cond', 'garage_qual', 'garage_finish', 'garage_type']
frame_column_filler(df_test, garage_cols, 'No Garage', inplace=True)

In [13]:
df_test.isnull().sum().sort_values(ascending=False).head()

lot_frontage      160
garage_yr_blt      45
bsmtfin_type_2     25
bsmtfin_type_1     25
bsmt_exposure      25
dtype: int64

I'm going to drop the `Garage Yr Blt` now and see how well we can do without it. I would think that newer garages would maybe be menaingful, but I'm not sure how to encode this column for those houses without a garage.

In [14]:
df_test.drop('garage_yr_blt', axis=1, inplace=True)

In [15]:
basement_cols = ['bsmt_exposure', 'bsmtfin_type_2', 'bsmt_cond', 'bsmt_qual','bsmtfin_type_1']
frame_column_filler(df_test, basement_cols, 'No Basement', inplace=True)

In [16]:
df_test.isnull().sum().sort_values(ascending=False).head(10)

lot_frontage     160
mas_vnr_area       1
garage_finish      1
electrical         1
garage_qual        1
garage_cond        1
mas_vnr_type       1
sale_type          0
roof_style         0
roof_matl          0
dtype: int64

The final rows that I believe are true errors in the missing values are the ones below `Lot Frontage`. We'll drop those rows now, then impute values for `Lot Frontage` based on `MS SubClass`.

In [18]:
medians = df_train[['lot_frontage', 'ms_subclass']].groupby('ms_subclass').agg(np.median).copy()
medians = {x:y[0] for x,y in zip(medians.index, medians.values)}

In [20]:
for index, lf, ms_sub in df_test[['lot_frontage', 'ms_subclass']].to_records():
    if np.isnan(lf):
#         print(ms_sub)
        df_test.at[index, 'lot_frontage'] = medians[ms_sub]


In [22]:
df_test.isnull().sum().sort_values(ascending=False).head(10)

mas_vnr_area     1
garage_finish    1
electrical       1
garage_qual      1
garage_cond      1
mas_vnr_type     1
sale_type        0
roof_style       0
roof_matl        0
exterior_1st     0
dtype: int64

Let's find the rows that need to be fixed still.

In [23]:
df_test[df_test.isnull().sum(1).astype('bool')]

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
id,,,,,,,,,,,,,,,,,,,,,
1578,80,RL,73.0,9735,Pave,No Alley,Reg,Lvl,AllPub,Inside,...,0,0,0,No Pool,No Fence,No Misc Feature,0,5,2008,WD
1357,60,RM,57.0,8094,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,0,No Pool,MnPrv,Shed,1000,9,2008,WD
868,60,RL,70.0,8749,Pave,No Alley,Reg,Lvl,AllPub,Inside,...,0,0,0,No Pool,No Fence,No Misc Feature,0,11,2009,WD


In [24]:
df_test.loc[1578].isnull().sum()

1

In [25]:
df_test.at[1578, 'electrical']

nan

In [26]:
df_train.electrical.value_counts()

SBrkr    1835
FuseA     138
FuseF      35
FuseP       7
Mix         1
Name: electrical, dtype: int64

In [27]:
df_test.at[1578, 'electrical'] = 'SBrkr'

In [28]:
df_test.loc[868].isnull().sum()

2

In [29]:
df_test.loc[868]['mas_vnr_area']

nan

In [30]:
df_test.loc[868]['mas_vnr_type']

nan

In [31]:
df_train['mas_vnr_area'].value_counts().head()

0.0      1211
120.0      11
200.0      10
176.0      10
72.0        9
Name: mas_vnr_area, dtype: int64

In [32]:
df_train['mas_vnr_type'].value_counts().head()

None       1208
BrkFace     627
Stone       168
BrkCmn       13
Name: mas_vnr_type, dtype: int64

In [33]:
df_test.at[868,'mas_vnr_area'] = 0
df_test.at[868,'mas_vnr_type'] = 'None'

In [34]:
df_test.loc[1357].isnull().sum()

3

In [35]:
for col in df_test.columns:
    if 'garage' in col:
        print(col, df_test.at[1357,col])

garage_type Detchd
garage_finish nan
garage_cars 1.0
garage_area 360.0
garage_qual nan
garage_cond nan


In [36]:
df_train[(df_train['garage_type']=='Detchd') & (df_train['garage_cars'] == 1)]['garage_finish'].value_counts()

Unf    243
RFn      5
Fin      2
Name: garage_finish, dtype: int64

In [37]:
df_train[(df_train['garage_type']=='Detchd') & (df_train['garage_cars'] == 1)]['garage_qual'].value_counts()

TA    201
Fa     44
Ex      2
Po      2
Gd      1
Name: garage_qual, dtype: int64

In [38]:
df_train[(df_train['garage_type']=='Detchd') & (df_train['garage_cars'] == 1)]['garage_cond'].value_counts()

TA    217
Fa     27
Po      4
Ex      2
Name: garage_cond, dtype: int64

In [39]:
for col in df_test.columns:
    if 'garage' in col:
        if pd.isnull(df_test.at[1357,col]):
            print(col, df_test.at[1357,col])

garage_finish nan
garage_qual nan
garage_cond nan


In [40]:
for col in df_test.columns:
    if 'garage' in col:
        if pd.isnull(df_test.at[1357,col]):
            print(col, df_test.at[1357,col])
            df_test.at[1357,col] = df_train[(df_train['garage_type']=='Detchd') & \
                                 (df_train['garage_cars'] == 1)][col].value_counts().index[0]
            print(col, df_test.at[1357,col])

garage_finish nan
garage_finish Unf
garage_qual nan
garage_qual TA
garage_cond nan
garage_cond TA


In [41]:
df_test.isnull().sum().sum()

0

No more null data at least, maybe I made bad choices about imputing, we shall see.

In [42]:
df_test.to_csv('../../data/cleaned/ames_clean_test.csv')

There could still be lots of outliers or other just wrong data that needs to be removed. Hopefully this will be discovered in EDA.